# Generate with NanoGPT

Here we are going to take our trained model and generate new text with it.

First lets do some imports:

In [1]:
import os
import torch
import pickle
import tiktoken

from contextlib import nullcontext

from nanoGPT.model import GPT
from nanoGPT.config import ModelConfig

### Generation hyperparameters
If you are on an M1 macbook or have an NVIDIA GPU, you may want to change the device from cpu to `mps` or `cuda` for faster training.

In [2]:
init_from = 'resume' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
ckpt_dir = 'ckpt/shakespeare-char' # ignored if init_from is not 'resume'
num_samples = 10 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cpu' # examples: 'cpu' (standard), 'cuda' (NVIDIA GPU), 'mps' (Mac M1/M2/M3)
dtype = 'float16' 
compile = False # use PyTorch 2.0 to compile the model to be faster
ctx = nullcontext()

### Start seed for text

This string will be the starting point for our text. 

If you want something longer you can load in a text file to condition the model, by adding the prefix: ` "FILE:prompt.txt"`.

In [3]:
start = "Once upon a time" # Can also specify a file

### Load model and generate

In [4]:
# model
if init_from == 'resume':
    # init from a model saved in a specific directory
    ckpt_path = os.path.join(ckpt_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    checkpoint_model_args = checkpoint['model_args']
    m = ModelConfig.from_dict(checkpoint_model_args)
    model = GPT(m)
    state_dict = checkpoint['model']
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
elif init_from.startswith('gpt2'):
    # init from a given GPT-2 model
    model = GPT.from_pretrained(init_from, dict(dropout=0.0))

model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

# look for the meta pickle in case it is available in the dataset folder
load_meta = False
if init_from == 'resume' and 'dataset' in checkpoint: # older checkpoints might not have these...
    meta_path = os.path.join(checkpoint['dataset'], 'meta.pkl')
    load_meta = os.path.exists(meta_path)
if load_meta:
    print(f"Loading meta from {meta_path}...")
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    # TODO want to make this more general to arbitrary encoder/decoder schemes
    stoi, itos = meta['stoi'], meta['itos']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])
else:
    # ok let's assume gpt-2 encodings by default
    print("No meta.pkl found, assuming GPT-2 encodings...")
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)

# encode the beginning of the prompt
if start.startswith('FILE:'):
    with open(start[5:], 'r', encoding='utf-8') as f:
        start = f.read()
start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

# run generation
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(decode(y[0].tolist()))
            print('---------------')


number of parameters: 10.65M
Loading meta from ../data/class-datasets/text-datasets/shakespeare_chars/meta.pkl...
Once upon a time brand thel hew?

MOFLOUS:
DOUS:
Herer tim deral my ony tend.
Havay alld by patatheis weshoutite th u be;
Gedou d tan! cusemiserthace in wis, h ll ethes mour sut nt; belllo Ones arthind ale
And tin chas or teabearsthin rstth tag me: gand wousind gr bengevet tis f br.


MEOLONCK:
Domer t ped bellin le, gher, wil'sthas r nede andd,
Ano st nd thor buth sthe, aroten:
Ase beangs frtove choug ath blele; o tot t te poret sthethe tho he anome hof acte toalelich, headoutonin

Lit t mo this asendoo st
Qul
---------------
Once upon a time thas, sis to sovest ill toin mom orh coune ais wis
Here lelld the blo the om-now ot-sth sealste
Dew ma couce nth henow wnize w ce sone somanthe olll.
 Gemin r owithsere Hang
Gon d d, weay, is nouth I t mest! gherd nofl an.


GLORY:
So fe itowand he the hirer wiratove h s cor ling thenrsthe asrenghe

I o wat we t by shee the cyord whiu

KeyboardInterrupt: 